In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder

# Definir el modelo MLP
def crear_modelo():
    modelo = Sequential()
    modelo.add(Dense(64, input_dim=3, activation='relu'))  # 3 características (promedio de R, G, B)
    modelo.add(Dense(32, activation='relu'))
    modelo.add(Dense(3, activation='softmax'))  # 3 clases: rojo, amarillo, verde
    modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return modelo

# Preprocesar la imagen para extraer los colores predominantes
def obtener_color_promedio(frame):
    # Convertimos el frame a espacio de color HSV para facilitar la detección de colores
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Definir rangos de colores en HSV para rojo, amarillo y verde
    rango_rojo = ((0, 100, 100), (10, 255, 255))  # Rojo
    rango_amarillo = ((20, 100, 100), (40, 255, 255))  # Amarillo
    rango_verde = ((40, 100, 100), (80, 255, 255))  # Verde

    # Máscara de colores
    rojo = cv2.inRange(hsv, rango_rojo[0], rango_rojo[1])
    amarillo = cv2.inRange(hsv, rango_amarillo[0], rango_amarillo[1])
    verde = cv2.inRange(hsv, rango_verde[0], rango_verde[1])

    # Calcular los promedios de los colores
    promedio_rojo = cv2.countNonZero(rojo)
    promedio_amarillo = cv2.countNonZero(amarillo)
    promedio_verde = cv2.countNonZero(verde)

    return np.array([promedio_rojo, promedio_amarillo, promedio_verde])

# Entrenar la red neuronal con datos de entrenamiento (simulados o reales)
def entrenar_red():
    # Para este ejemplo, usamos datos simulados (promedios de colores y etiquetas)
    # Datos de entrenamiento (promedio de los colores R, G, B de cada semáforo)
    X_train = np.array([
        [1500, 0, 0],   # Rojo
        [0, 1000, 0],   # Amarillo
        [0, 0, 2000],   # Verde
        [1400, 0, 100],  # Rojo
        [0, 1200, 100],  # Amarillo
        [0, 0, 1800]    # Verde
    ])

    # Etiquetas correspondientes (0: Rojo, 1: Amarillo, 2: Verde)
    y_train = np.array([0, 1, 2, 0, 1, 2])

    # Convertir las etiquetas a formato One-Hot
    encoder = LabelEncoder()
    y_train_encoded = encoder.fit_transform(y_train)
    y_train_onehot = tf.keras.utils.to_categorical(y_train_encoded, 3)

    # Crear y entrenar el modelo
    modelo = crear_modelo()
    modelo.fit(X_train, y_train_onehot, epochs=50, batch_size=10, verbose=1)

    return modelo

# Función para predecir el color del semáforo en tiempo real
def predecir_color(frame, modelo):
    # Obtener los promedios de los colores
    promedio_colores = obtener_color_promedio(frame)

    # Hacer una predicción usando el modelo
    prediccion = modelo.predict(np.array([promedio_colores]))
    clase_predicha = np.argmax(prediccion)  # Obtenemos la clase con mayor probabilidad

    if clase_predicha == 0:
        return "Rojo"
    elif clase_predicha == 1:
        return "Amarillo"
    elif clase_predicha == 2:
        return "Verde"

# Iniciar la cámara y usar el modelo para predecir el color del semáforo
def iniciarCamara(modelo):
    print("Iniciando módulo de cámara...")
    cam = cv2.VideoCapture(0)

    if not cam.isOpened():
        print("No se pudo abrir la cámara")
        return

    print("Cámara abierta")

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Error al capturar imagen")
            break

        # Predecir el color del semáforo en la imagen
        color_predicho = predecir_color(frame, modelo)
        print(f"Color del semáforo: {color_predicho}")

        # Mostrar el resultado
        cv2.putText(frame, f"Color: {color_predicho}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Camara", frame)  # Mostrar la imagen con la predicción

        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # Si presionamos 'Esc', salimos
            break

    cam.release()
    cv2.destroyAllWindows()

# Este bloque previene que se ejecute dos veces si lo importas en otro script
if __name__ == "__main__":
    # Entrenamos el modelo
    modelo = entrenar_red()
    # Iniciamos la cámara
    iniciarCamara(modelo)

C:\Users\Equipo\PycharmProjects\JupyterProject\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0000e+00 - loss: 151.7201
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.1667 - loss: 125.3579
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.3333 - loss: 107.6123
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.3333 - loss: 94.8919
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.3333 - loss: 79.8726
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.3333 - loss: 65.2989
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.3333 - loss: 51.5975
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.3333 - loss: 37.3828
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.3333 - loss: 23.2756
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.6667 - loss: 16.5725
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.6667 - loss: 11.7756
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6667 

In [ ]:
import cv2
import numpy as np

# Clase para la red neuronal de Hopfield
class RedHopfield:
    def __init__(self, num_neuronas):
        self.num_neuronas = num_neuronas
        self.pesos = np.zeros((num_neuronas, num_neuronas))

    def entrenar(self, patrones):
        for patron in patrones:
            patron = np.where(patron == 255, 1, -1)
            self.pesos += np.outer(patron, patron)
        np.fill_diagonal(self.pesos, 0)

    def reconocer(self, entrada):
        salida = entrada.copy()
        for _ in range(10):
            salida = np.sign(np.dot(self.pesos, salida))
            salida = np.where(salida == -1, 0, 255)
        return salida

# Función para convertir una imagen a un patrón binario después de redimensionarla
def convertir_a_patron(imagen, nuevo_tamaño=(32, 32)):
    imagen = cv2.resize(imagen, nuevo_tamaño)
    gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    _, binario = cv2.threshold(gris, 128, 255, cv2.THRESH_BINARY)
    return binario.flatten()

# Función para identificar el color principal de la imagen
def identificar_color(frame):
    # Convertimos la imagen a HSV (Espacio de color más adecuado para identificar colores)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Definimos los rangos para cada color (rojo, amarillo y verde)
    # Rojo
    rojo_bajo = np.array([0, 100, 100])
    rojo_alto = np.array([10, 255, 255])

    # Amarillo
    amarillo_bajo = np.array([20, 100, 100])
    amarillo_alto = np.array([40, 255, 255])

    # Verde
    verde_bajo = np.array([40, 100, 100])
    verde_alto = np.array([80, 255, 255])

    # Creamos las máscaras para cada color
    mascara_rojo = cv2.inRange(hsv, rojo_bajo, rojo_alto)
    mascara_amarillo = cv2.inRange(hsv, amarillo_bajo, amarillo_alto)
    mascara_verde = cv2.inRange(hsv, verde_bajo, verde_alto)

    # Sumamos los valores de la máscara para cada color (cuántos píxeles son de ese color)
    rojo_sum = np.sum(mascara_rojo)
    amarillo_sum = np.sum(mascara_amarillo)
    verde_sum = np.sum(mascara_verde)

    # Comparamos cuál tiene más píxeles de ese color
    if rojo_sum > amarillo_sum and rojo_sum > verde_sum:
        return "Rojo"
    elif amarillo_sum > rojo_sum and amarillo_sum > verde_sum:
        return "Amarillo"
    elif verde_sum > rojo_sum and verde_sum > amarillo_sum:
        return "Verde"
    else:
        return "Desconocido"

# Función para predecir el color basado en la imagen capturada
def predecir_color(frame):
    return identificar_color(frame)

# Función para iniciar la cámara y predecir el color
def iniciarCamara():
    print("Iniciando módulo")
    cam = cv2.VideoCapture(0)
    if not cam.isOpened():
        print("No se pudo abrir la cámara")
        return
    print("Cámara abierta")

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Error al encontrar imagen")
            break

        # Predecir el color usando la función de detección
        color_detectado = predecir_color(frame)

        # Mostrar el color detectado
        print(f"Color Detectado: {color_detectado}")

        # Advertir según el color detectado
        if color_detectado == "Rojo":
            print("¡Alerta! El semáforo está en rojo.")
        elif color_detectado == "Amarillo":
            print("¡Precaución! El semáforo está en amarillo.")
        elif color_detectado == "Verde":
            print("¡Verde! Puedes avanzar.")
        else:
            print("Color Desconocido")

        # Mostrar la imagen de la cámara
        cv2.imshow("Cámara", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # Esc para salir
            break

    cam.release()
    cv2.destroyAllWindows()

# Ejecutar la cámara
if __name__ == "__main__":
    iniciarCamara()

Iniciando módulo
Cámara abierta
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido
Color Desconocido
Color Detectado: Desconocido

In [ ]:
import numpy as np
import cv2

# Función para convertir una imagen a un patrón binario después de redimensionarla
def convertir_a_patron(imagen, nuevo_tamaño=(32, 32)):
    # Redimensionar la imagen a un tamaño más pequeño
    imagen = cv2.resize(imagen, nuevo_tamaño)  # Redimensionar a 32x32 píxeles
    # Convertir a escala de grises
    gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    # Convertir a un patrón binario (0 o 255)
    _, binario = cv2.threshold(gris, 128, 255, cv2.THRESH_BINARY)
    # Aplanar la imagen binaria en un vector
    return binario.flatten()

# Clase para la red neuronal de Hopfield
class RedHopfield:
    def __init__(self, num_neuronas):
        self.num_neuronas = num_neuronas
        # Inicializamos la matriz de pesos con ceros
        self.pesos = np.zeros((num_neuronas, num_neuronas))

    def entrenar(self, patrones):
        # Entrenar la red con los patrones
        for patron in patrones:
            patron = np.where(patron == 255, 1, -1)  # Convertimos 255 a +1 y 0 a -1
            self.pesos += np.outer(patron, patron)
        np.fill_diagonal(self.pesos, 0)  # Eliminamos la conexión con la misma neurona

    def reconocer(self, entrada):
        # Propagar el patrón a través de la red
        salida = entrada.copy()
        for _ in range(10):  # Número de iteraciones
            salida = np.sign(np.dot(self.pesos, salida))  # Calculamos la salida
            salida = np.where(salida == -1, 0, 255)  # Convertimos de nuevo a 0 y 255
        return salida

# Función para entrenar la red con las imágenes del semáforo
def entrenar_red_semaforo():
    # Cargar imágenes representativas para cada color
    rojo = cv2.imread("imagenes/rojo1.jpg")
    amarillo = cv2.imread("imagenes/amarillo1.jpg")
    verde = cv2.imread("imagenes/verde1.jpg")

    # Convertir las imágenes a patrones binarios (después de redimensionarlas)
    patron_rojo = convertir_a_patron(rojo)
    patron_amarillo = convertir_a_patron(amarillo)
    patron_verde = convertir_a_patron(verde)

    # Crear la red de Hopfield con el tamaño adecuado (número de neuronas)
    red = RedHopfield(len(patron_rojo))  # El número de neuronas es el tamaño del patrón

    # Entrenar la red con los tres patrones
    red.entrenar([patron_rojo, patron_amarillo, patron_verde])

    # Retornar la red entrenada
    return red

# Entrenar la red
red_semaforo = entrenar_red_semaforo()

# Guardar el modelo entrenado (pesos de la red)
np.save("entrenamientos_guardados/pesos_red_semaforo.npy", red_semaforo.pesos)

print("Entrenamiento completado y pesos guardados como 'pesos_red_semaforo.npy'")


In [ ]:
import cv2
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import joblib
import os


# Función para extraer características del color dominante
def extract_color_features(image):
    # Convertir a HSV que es mejor para detección de color
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Calcular histograma de los canales H (tono) y S (saturación)
    hist_h = cv2.calcHist([hsv], [0], None, [180], [0, 180])
    hist_s = cv2.calcHist([hsv], [1], None, [256], [0, 256])

    # Normalizar histogramas
    hist_h = cv2.normalize(hist_h, hist_h).flatten()
    hist_s = cv2.normalize(hist_s, hist_s).flatten()

    # Concatenar características
    features = np.hstack([hist_h, hist_s])
    return features


# Función para generar datos de entrenamiento
def generate_training_data():
    colors = ['rojo', 'amarillo', 'verde']
    X = []
    y = []

    print("Generando datos de entrenamiento...")

    for color_idx, color in enumerate(colors):
        print(f"Procesando imágenes para {color}...")

        for _ in range(100):
            if color == 'rojo':
                # Valores HSV para rojo
                h = np.random.randint(0, 10)  # Rojo está en ambos extremos del espectro
                s = np.random.randint(150, 255)
                v = np.random.randint(150, 255)
            elif color == 'amarillo':
                # Valores HSV para amarillo
                h = np.random.randint(20, 40)
                s = np.random.randint(150, 255)
                v = np.random.randint(150, 255)
            else:  # verde
                # Valores HSV para verde
                h = np.random.randint(50, 70)
                s = np.random.randint(150, 255)
                v = np.random.randint(150, 255)

            # Crear imagen sintética
            synthetic_img = np.zeros((32, 32, 3), dtype=np.uint8)
            synthetic_img[:, :] = [h, s, v]
            synthetic_img = cv2.cvtColor(synthetic_img, cv2.COLOR_HSV2BGR)

            # Extraer características
            features = extract_color_features(synthetic_img)
            X.append(features)
            y.append(color_idx)

    return np.array(X), np.array(y)


# Entrenar el modelo MLP
def train_model():
    X, y = generate_training_data()

    # Dividir datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Escalar datos
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Crear y entrenar MLP
    mlp = MLPClassifier(
        hidden_layer_sizes=(70, 40),
        activation='relu',
        solver='adam',
        max_iter=10000,
        random_state=42,
        verbose=True
    )

    print("Entrenando modelo...")
    mlp.fit(X_train, y_train)

    # Evaluar modelo
    y_pred = mlp.predict(X_test)
    print(classification_report(y_test, y_pred))

    # Guardar modelo y scaler
    joblib.dump(mlp, 'entrenamientos_guardados/semaforo_mlp_model.pkl')
    joblib.dump(scaler, 'entrenamientos_guardados/semaforo_scaler.pkl')
    print("Modelo guardado correctamente.")


if __name__ == "__main__":
    train_model()
import cv2
import numpy as np
import joblib
from time import sleep

# Cargar modelo y scaler en el ámbito global
try:
    mlp = joblib.load('entrenamientos_guardados/semaforo_mlp_model.pkl')
    scaler = joblib.load('entrenamientos_guardados/semaforo_scaler.pkl')
    print("Modelo cargado correctamente.")
except Exception as e:
    print(f"Error al cargar el modelo: {e}")
    print("Asegúrate de haber ejecutado primero el script de entrenamiento.")
    exit()

# Mapeo de clases a colores
color_map = {
    0: ('ROJO', (0, 0, 255), "PELIGRO - Detengase"),
    1: ('AMARILLO', (0, 255, 255), "PRECAUCION - Reduzca velocidad"),
    2: ('VERDE', (0, 255, 0), "AVANCE - Puede continuar")
}


def extract_color_features(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist_h = cv2.calcHist([hsv], [0], None, [180], [0, 180])
    hist_s = cv2.calcHist([hsv], [1], None, [256], [0, 256])
    hist_h = cv2.normalize(hist_h, hist_h).flatten()
    hist_s = cv2.normalize(hist_s, hist_s).flatten()
    return np.hstack([hist_h, hist_s])


def predict_color(frame):
    global mlp, scaler  # Acceder a las variables globales
    features = extract_color_features(frame)
    features_scaled = scaler.transform([features])
    pred = mlp.predict(features_scaled)[0]
    return pred


def iniciar_semaforo_inteligente():
    print("Iniciando semáforo inteligente...")
    cam = cv2.VideoCapture(0)

    if not cam.isOpened():
        print("No se pudo abrir la cámara")
        return

    cv2.namedWindow("Semáforo Inteligente", cv2.WINDOW_NORMAL)

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Error al capturar imagen")
            break

        frame_small = cv2.resize(frame, (320, 240))
        color_pred = predict_color(frame_small)

        try:
            color_name, color_bgr, mensaje = color_map[color_pred]
        except KeyError:
            print(f"Predicción inválida: {color_pred}")
            continue

        cv2.putText(frame, f"Color detectado: {color_name}", (20, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color_bgr, 2)
        cv2.putText(frame, mensaje, (20, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color_bgr, 2)

        cv2.rectangle(frame, (20, 100), (100, 280), (255, 255, 255), 2)

        # Luz roja
        cv2.circle(frame, (60, 140), 30, (0, 0, 255) if color_pred == 0 else (50, 50, 50), -1)
        # Luz amarilla
        cv2.circle(frame, (60, 190), 30, (0, 255, 255) if color_pred == 1 else (50, 50, 50), -1)
        # Luz verde
        cv2.circle(frame, (60, 240), 30, (0, 255, 0) if color_pred == 2 else (50, 50, 50), -1)

        cv2.imshow("Semáforo Inteligente", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == 27:
            break

    cam.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    iniciar_semaforo_inteligente()

In [ ]:
# entrenar_rbf.py
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.linear_model import LogisticRegression
import joblib


# Generar más muestras sintéticas por clase
def generar_colores(base_color, variacion=20, n=30):
    return np.clip(
        np.random.normal(loc=base_color, scale=variacion, size=(n, 3)),
        0, 255
    )


rojos = generar_colores([200, 0, 0])
amarillos = generar_colores([255, 255, 0])
verdes = generar_colores([0, 200, 0])

X = np.vstack((rojos, amarillos, verdes))
y = np.array([0] * len(rojos) + [1] * len(amarillos) + [2] * len(verdes))

# Clustering para centros RBF
kmeans = KMeans(n_clusters=6, random_state=42).fit(X)
centros = kmeans.cluster_centers_

X_rbf = rbf_kernel(X, centros, gamma=1e-4)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_rbf, y)

joblib.dump(centros, "entrenamientos_guardados/centros_rbf.pkl")
joblib.dump(clf, "entrenamientos_guardados/clasificador_rbf.pkl")

print("✅ Entrenamiento sintético finalizado correctamente.")

# semaforo_rbf.py
import cv2
import numpy as np
import joblib
from sklearn.metrics.pairwise import rbf_kernel


def obtener_color_dominante(frame):
    resized = cv2.resize(frame, (50, 50))
    data = resized.reshape((-1, 3))
    promedio = np.mean(data, axis=0)
    return promedio.reshape(1, -1), promedio.astype(int)


def interpretar_clase(clase):
    if clase == 0:
        return "ALTO: ROJO", (0, 0, 255)
    elif clase == 1:
        return "PRECAUCIÓN: AMARILLO", (0, 255, 255)
    elif clase == 2:
        return "ADELANTE: VERDE", (0, 255, 0)
    else:
        return "DESCONOCIDO", (255, 255, 255)


def iniciarCamara():
    cam = cv2.VideoCapture(0)
    if not cam.isOpened():
        print("No se pudo abrir la cámara")
        return

    # Cargar modelo
    centros = joblib.load("entrenamientos_guardados/centros_rbf.pkl")
    clf = joblib.load("entrenamientos_guardados/clasificador_rbf.pkl")

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Error al capturar imagen")
            break

        color_dom, color_rgb = obtener_color_dominante(frame)
        rbf_input = rbf_kernel(color_dom, centros, gamma=1e-4)
        clase = clf.predict(rbf_input)[0]
        mensaje, color_texto = interpretar_clase(clase)

        # Visualización del color dominante
        cv2.rectangle(frame, (0, 0), (150, 50), tuple(int(c) for c in color_rgb), -1)
        cv2.putText(frame, mensaje, (160, 35),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color_texto, 2)

        cv2.imshow("Semáforo Inteligente", frame)

        if cv2.waitKey(1) & 0xFF == 27:  # ESC
            break

    cam.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    iniciarCamara()


In [ ]:
import cv2
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import joblib
from collections import Counter

# Definición de colores global
COLORS = {
    'rojo': {'h_range': (0, 10), 's_range': (150, 255), 'v_range': (150, 255)},
    'amarillo': {'h_range': (20, 40), 's_range': (150, 255), 'v_range': (150, 255)},
    'verde': {'h_range': (50, 70), 's_range': (150, 255), 'v_range': (150, 255)}
}


def detect_dominant_colors(image, k=3):
    """Detecta los k colores predominantes en la imagen usando k-means"""
    pixels = np.float32(image.reshape(-1, 3))

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, 0.2)
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    centers_uint8 = np.uint8([centers])
    centers_hsv = cv2.cvtColor(centers_uint8, cv2.COLOR_BGR2HSV)[0]

    counts = Counter(labels.flatten())
    total = sum(counts.values())

    dominant_colors = []
    for i, (color_bgr, color_hsv) in enumerate(zip(centers, centers_hsv)):
        percentage = counts[i] / total
        dominant_colors.append({
            'bgr': color_bgr,
            'hsv': color_hsv,
            'percentage': percentage
        })

    return sorted(dominant_colors, key=lambda x: -x['percentage'])


def generate_training_data():
    X = []
    y = []

    print("Generando datos de entrenamiento avanzado...")

    for color_idx, (color_name, color_params) in enumerate(COLORS.items()):
        print(f"Procesando color: {color_name}")

        for _ in range(300):
            img_hsv = np.zeros((64, 64, 3), dtype=np.uint8)

            h_main = np.random.randint(*color_params['h_range'])
            s_main = np.random.randint(*color_params['s_range'])
            v_main = np.random.randint(*color_params['v_range'])

            for _ in range(np.random.randint(2, 5)):
                h_noise = np.random.randint(0, 180)
                s_noise = np.random.randint(50, 255)
                v_noise = np.random.randint(50, 255)

                mask = np.random.rand(64, 64) > 0.7
                img_hsv[mask] = [h_noise, s_noise, v_noise]

            y1, y2 = np.random.randint(0, 44), np.random.randint(20, 64)
            x1, x2 = np.random.randint(0, 44), np.random.randint(20, 64)
            img_hsv[y1:y2, x1:x2] = [h_main, s_main, v_main]

            img_bgr = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)
            dominant_colors = detect_dominant_colors(img_bgr)

            dominant_hsv = dominant_colors[0]['hsv']
            features = [dominant_hsv[0], dominant_hsv[1], dominant_hsv[2], dominant_colors[0]['percentage']]

            X.append(features)
            y.append(color_idx)

    return np.array(X), np.array(y)


def train_model():
    X, y = generate_training_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    mlp = MLPClassifier(
        hidden_layer_sizes=(128, 64, 32),
        activation='relu',
        solver='adam',
        max_iter=1000,
        early_stopping=True,
        validation_fraction=0.2,
        random_state=42,
        verbose=True
    )

    print("Entrenando modelo avanzado...")
    mlp.fit(X_train, y_train)

    y_pred = mlp.predict(X_test)
    print("\nReporte de clasificación:")
    print(classification_report(y_test, y_pred, target_names=list(COLORS.keys())))

    joblib.dump(mlp, 'entrenamientos_guardados/semaforo_mlp_model.pkl')
    joblib.dump(scaler, 'entrenamientos_guardados/semaforo_scaler.pkl')
    print("Modelo avanzado guardado correctamente.")


if __name__ == "__main__":
    train_model()
import cv2
import numpy as np
import joblib
from collections import Counter

# Cargar modelo y scaler
try:
    mlp = joblib.load('entrenamientos_guardados/semaforo_mlp_model.pkl')
    scaler = joblib.load('entrenamientos_guardados/semaforo_scaler.pkl')
    print("Modelo cargado correctamente.")
except Exception as e:
    print(f"Error al cargar el modelo: {e}")
    exit()

COLOR_MAP = {
    0: ('ROJO', (0, 0, 255), "PELIGRO - Deténgase"),
    1: ('AMARILLO', (0, 255, 255), "PRECAUCIÓN - Reduzca velocidad"),
    2: ('VERDE', (0, 255, 0), "AVANCE - Puede continuar")
}


def detect_dominant_colors(image, k=3):
    pixels = np.float32(image.reshape(-1, 3))
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, 0.2)
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    centers_uint8 = np.uint8([centers])
    centers_hsv = cv2.cvtColor(centers_uint8, cv2.COLOR_BGR2HSV)[0]

    counts = Counter(labels.flatten())
    total = sum(counts.values())

    dominant_colors = []
    for i, (color_bgr, color_hsv) in enumerate(zip(centers, centers_hsv)):
        percentage = counts[i] / total
        dominant_colors.append({
            'bgr': color_bgr,
            'hsv': color_hsv,
            'percentage': percentage
        })

    return sorted(dominant_colors, key=lambda x: -x['percentage'])


def predict_dominant_traffic_color(frame):
    small_frame = cv2.resize(frame, (320, 240))
    dominant_colors = detect_dominant_colors(small_frame, k=5)

    for color_info in dominant_colors:
        h, s, v = color_info['hsv']
        percentage = color_info['percentage']

        if s < 100 or v < 100 or percentage < 0.1:
            continue

        features = np.array([[h, s, v, percentage]])
        features_scaled = scaler.transform(features)

        pred = mlp.predict(features_scaled)[0]
        confidence = np.max(mlp.predict_proba(features_scaled))

        if confidence > 0.8:
            return pred, color_info, confidence

    return None, None, 0


def draw_traffic_light(frame, active_color):
    cv2.rectangle(frame, (20, 20), (100, 200), (255, 255, 255), 2)
    colors = {0: (0, 0, 255), 1: (0, 255, 255), 2: (0, 255, 0)}
    for i, y in enumerate([60, 110, 160]):
        color = colors[i] if i == active_color else (50, 50, 50)
        cv2.circle(frame, (60, y), 25, color, -1)


def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error al abrir la cámara")
        return

    cv2.namedWindow("Semaforo Inteligente", cv2.WINDOW_NORMAL)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error al capturar frame")
            break

        pred, color_info, confidence = predict_dominant_traffic_color(frame)

        if pred is not None:
            color_name, text_color, message = COLOR_MAP[pred]
            draw_traffic_light(frame, pred)

            cv2.putText(frame, f"Color: {color_name}", (120, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, text_color, 2)
            cv2.putText(frame, message, (120, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, text_color, 2)
            cv2.putText(frame, f"Confianza: {confidence:.2f}", (120, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

            if color_info:
                bgr_color = tuple(map(int, color_info['bgr']))
                cv2.rectangle(frame, (120, 170), (170, 200), bgr_color, -1)

        cv2.imshow("Semaforo Inteligente", frame)

        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()